In [1]:
# Set the seed value for the notebook so the results are reproducible
from numpy.random import seed
seed(1)

In [2]:
# Dependencies
import numpy as np
import pandas as pd

In [3]:
import tensorflow
tensorflow.keras.__version__

'2.2.4-tf'

In [4]:
df = pd.read_csv("main_data_tree.csv")
df.head(10)

,EOG_quintile,percent_EDS,child_abuse_rate,children_conc_pov,elevated_lead,juv_delinquent,No_HSdegree,parent_unemployed,county_poc,county_tier,poverty_county,median_inc_county
0,4,96.0,4.2,21.2,1.7,13.7,13.5,11.3,50.3,1,37.2,40433
1,4,96.0,6.3,18.9,1.3,11.5,6.3,8.3,21.6,3,20.0,53419
2,3,96.0,2.0,20.5,0.9,9.9,11.5,8.6,56.6,3,18.9,59329
3,4,96.0,9.9,39.3,1.4,19.1,19.8,19.3,74.2,1,34.7,35407
4,4,4.0,1.1,27.9,1.5,23.3,9.8,9.5,42.7,2,23.0,50112
5,1,4.0,3.0,0.0,2.6,22.8,10.2,2.8,11.5,3,12.4,64426
6,1,4.0,2.2,22.5,1.9,18.4,16.7,12.7,52.5,1,29.6,44015
7,1,4.0,1.8,3.1,1.2,6.7,5.9,4.0,39.1,3,9.7,80169
8,3,4.0,0.8,11.8,1.0,4.8,12.6,9.2,35.7,2,18.7,57710
9,1,4.0,1.8,3.1,1.2,6.7,5.9,4.0,39.1,3,9.7,80169


## Data Pre-Processing

In [5]:
X = df.drop("EOG_quintile", axis=1)
y = df["EOG_quintile"]
print(X.shape, y.shape)

(2492, 11) (2492,)


In [6]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical

In [7]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=1, stratify=y)

In [8]:
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [9]:
# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

In [10]:
# Step 2: Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

# Create a Deep Learning Model

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [12]:
# Create model and add layers
model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=11))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=5, activation='softmax'))

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [13]:
# Compile and fit the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               1200      
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 505       
Total params: 11,805
Trainable params: 11,805
Non-trainable params: 0
_________________________________________________________________


In [15]:
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=60,
    shuffle=True,
    verbose=2
)

Epoch 1/60
1869/1869 - 0s - loss: 1.3288 - acc: 0.4462
Epoch 2/60
1869/1869 - 0s - loss: 1.1466 - acc: 0.5254
Epoch 3/60
1869/1869 - 0s - loss: 1.0761 - acc: 0.5447
Epoch 4/60
1869/1869 - 0s - loss: 1.0454 - acc: 0.5457
Epoch 5/60
1869/1869 - 0s - loss: 1.0246 - acc: 0.5618
Epoch 6/60
1869/1869 - 0s - loss: 1.0226 - acc: 0.5629
Epoch 7/60
1869/1869 - 0s - loss: 1.0032 - acc: 0.5709
Epoch 8/60
1869/1869 - 0s - loss: 1.0019 - acc: 0.5811
Epoch 9/60
1869/1869 - 0s - loss: 1.0021 - acc: 0.5639
Epoch 10/60
1869/1869 - 0s - loss: 0.9928 - acc: 0.5752
Epoch 11/60
1869/1869 - 0s - loss: 0.9877 - acc: 0.5762
Epoch 12/60
1869/1869 - 0s - loss: 0.9858 - acc: 0.5778
Epoch 13/60
1869/1869 - 0s - loss: 0.9811 - acc: 0.5821
Epoch 14/60
1869/1869 - 0s - loss: 0.9805 - acc: 0.5773
Epoch 15/60
1869/1869 - 0s - loss: 0.9755 - acc: 0.5853
Epoch 16/60
1869/1869 - 0s - loss: 0.9735 - acc: 0.5848
Epoch 17/60
1869/1869 - 0s - loss: 0.9687 - acc: 0.5912
Epoch 18/60
1869/1869 - 0s - loss: 0.9656 - acc: 0.5875
E

## Quantify our Trained Model

In [16]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

623/623 - 0s - loss: 0.9160 - acc: 0.6164
Normal Neural Network - Loss: 0.9160256314048032, Accuracy: 0.6163724064826965


## Make Predictions

In [17]:
encoded_predictions = model.predict_classes(X_test_scaled[:20])
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

In [18]:
print(f"Predicted classes: {prediction_labels}")
print(f"Actual Labels: {list(y_test[:20])}")

Predicted classes: [3 3 3 3 2 2 2 3 4 2 2 2 3 1 2 3 2 3 1 3]
Actual Labels: [3, 3, 3, 3, 2, 1, 2, 3, 4, 2, 3, 2, 3, 1, 3, 3, 2, 3, 1, 3]
